In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB

*Создаем класс Баеса*

In [2]:
class Naive_Bayes():

    def __init__(self):
        pass

    def fit(self, X, y):

        # # Для использования в predict-ах
        # self.X = X
        # self.y = y

        categ = X.copy()
        categ[y.name] = y.values

        # Делаем подсчет уникальных элементов
        dict_counter = dict()
        for c in categ.columns:
            dict_counter[c] = Counter(categ[c])

        # print(dict_counter)

        # Подсчет вероятностей
        self.probabilities = dict()
        for column in list(dict_counter):
            summ = sum(dict_counter[column].values()) # Общее кол-во элементов (можно использовать кол-во записей)
            self.probabilities.update({k: v/summ  for k, v in dict_counter[column].items()})
        # print("probabilities: ", self.probabilities)

        # Расчет условных вероятностей
        x_property = np.unique(X.values.reshape(1,-1)) # найдем все значения свойств в X
        x_property = {x: np.unique(X[x]) for x in X}

        self.y_unique = np.unique(y)
        i = 0
        no_devide_zero = 0.01
        conditional_probability = dict()
        for y_v in self.y_unique:
            for x_k, x_v_list in x_property.items():
                for x_v in x_v_list:
                    i+=1
                    x_y_count = categ[categ[x_k] == x_v][categ[y.name] == y_v][y.name].size # Кол-во записей, где оба критерия выполняются
                    y_count = categ[categ[y.name] == y_v][y.name].size # кол-во записей, где выполняется только y
                    conditional_probability.update({x_v + "|" + y_v: x_y_count / (y_count + no_devide_zero)})
                    # name = y_v + "|" + x_v
                    # conditional_probability[name]= x_y_count / x_count

        # print(conditional_probability)
        # print("i",i)

        # Сохраняем вероятности в один словарь
        self.probabilities.update(conditional_probability)

    # Классификатор массива
    def predict(self, X):
        return [self.predict_proba(x) for x in X.values]

    # Классификатор
    def predict_proba(self, X):

        # predict_y = "Косяк"
        predict_probabilities = dict()
        # print(self.probabilities)
        for y in self.y_unique:
            keys_probabilities = [self.probabilities[f"{X[0]}|{y}"],
                                  self.probabilities[f"{X[1]}|{y}"],
                                  self.probabilities[f"{X[2]}|{y}"],
                                  self.probabilities[f"{X[3]}|{y}"],
                                  self.probabilities[y],
                                  self.probabilities[X[0]],
                                  self.probabilities[X[1]],
                                  self.probabilities[X[2]],
                                  self.probabilities[X[3]]]

            rezult = 1 # для умножения
            c = 0 # коэффициент
            for k in range(len(keys_probabilities)):
                if k < 5:
                    rezult *= keys_probabilities[k] + c
                    continue
                rezult /= keys_probabilities[k] + c
            predict_probabilities[y] = rezult

        # print(predict_probabilities)
        # print(sum(predict_probabilities.values()))

        return max(predict_probabilities, key=predict_probabilities.get)

    def score(self, X, y):
        rez = np.array(self.predict(X))
        return rez[y == rez].size / rez.size


In [3]:
def score_categorical_bayes(X, y):

    clf = CategoricalNB()

    # Перевод категориальных данных в численное представление
    from sklearn.preprocessing import OrdinalEncoder
    ord_enc = OrdinalEncoder()
    X = X.select_dtypes(include=['object']).copy() # для создания нового фрейма данных, содержащего только столбцы типа object
    y = y.astype(object).copy()

    X = ord_enc.fit_transform(X) # преобразуем объекты в число
    y = ord_enc.fit_transform(np.array(y).reshape(-1, 1)).ravel()

    # obj_df[obj_df.isnull().any(axis=1)] # Очистка нулевых значений

    # KFold
    from sklearn.model_selection import KFold

    # инициализация KFold с 5тью фолдами
    kf = KFold(n_splits=5)
    classifiers = []

    # метод split возвращает индексы для объектов train и test
    for train_index, test_index in kf.split(X):
        clf_cross = CategoricalNB()
        X_train, X_test = X[train_index[0]:train_index[-1]], X[test_index[0]:test_index[-1]]
        y_train, y_test = y[train_index[0]:train_index[-1]], y[test_index[0]:test_index[-1]]
        clf_cross.fit(X_train, y_train)
        classifiers.append(clf_cross.score(X_test, y_test)) # получим 5 оптимальных классификаторов
    print("KFold: ", classifiers) # Выводим рез-тат кроссвалидации

    # Делим данные на тестовые и тренировочные
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

    clf.fit(x_train, y_train)
    print("CategoricalNB() score: ", clf.score(x_test, y_test))

In [4]:
def score_my_naive_bayes(X, y):

    clf = Naive_Bayes()

    clf.fit(X, y)

    # KFold
    from sklearn.model_selection import KFold

    # инициализация KFold с 5тью фолдами
    kf = KFold(n_splits=5)
    classifiers = []

    # метод split возвращает индексы для объектов train и test
    for train_index, test_index in kf.split(X):
        clf_cross = Naive_Bayes()
        X_train, X_test = X[train_index[0]:train_index[-1]], X[test_index[0]:test_index[-1]]
        y_train, y_test = y[train_index[0]:train_index[-1]], y[test_index[0]:test_index[-1]]
        clf_cross.fit(X_train, y_train)
        classifiers.append(clf_cross.score(X_test, y_test)) # получим 5 оптимальных классификаторов
    print("KFold: ", classifiers) # Выводим рез-тат кроссвалидации

    # Делим данные на тестовые и тренировочные
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

    clf.fit(x_train, y_train)
    print("Naive_Baes score: ", clf.score(x_test, y_test) )

In [5]:
def main():
    # Загружаем датасеты
    for dts in ["./categ.csv", "./categ_2.csv"]:
        categ = pd.read_csv(dts, delimiter=",")
        del categ['Unnamed: 0'] # Удаляем столбец подсчета

        # Разделяем данные
        X = categ[["размер", "ткань", "цвет", "рукав"]]
        y = categ["спрос"]

        print("<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ", dts, "--------------------------------->\n")

        print("CategoricalNB from sklearn:")
        score_categorical_bayes(X, y)
        print()

        print("Naive_Baes from my script:")
        score_my_naive_bayes(X,y)
        print()

In [6]:
if __name__ == "__main__":
    main()

<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ ./categ.csv --------------------------------->

CategoricalNB from sklearn:
KFold:  [0.8164082041020511, 0.8199099549774888, 0.8194097048524263, 0.8084042021010506, 0.8104052026013007]
CategoricalNB() score:  0.8046666666666666

Naive_Baes from my script:


C:\Users\kuzin\AppData\Local\Temp/ipykernel_2912/3318508606.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x_y_count = categ[categ[x_k] == x_v][categ[y.name] == y_v][y.name].size # Кол-во записей, где оба критерия выполняются


KFold:  [0.8209104552276139, 0.8204102051025512, 0.8214107053526764, 0.8084042021010506, 0.8109054527263632]
Naive_Baes score:  0.805

<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ ./categ_2.csv --------------------------------->

CategoricalNB from sklearn:
KFold:  [0.7958979489744873, 0.7713856928464232, 0.7788894447223612, 0.7908954477238619, 0.7898949474737369]
CategoricalNB() score:  0.7946666666666666

Naive_Baes from my script:


C:\Users\kuzin\AppData\Local\Temp/ipykernel_2912/3318508606.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x_y_count = categ[categ[x_k] == x_v][categ[y.name] == y_v][y.name].size # Кол-во записей, где оба критерия выполняются


KFold:  [0.7968984492246123, 0.7738869434717359, 0.7808904452226113, 0.7943971985992997, 0.7913956978489245]
Naive_Baes score:  0.7923333333333333

